# Data Cleaning 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Datasets/Incidents_service.xlsx')
df.shape

(141712, 25)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
df_new = df[[ 'count_updated', 'opened_by', 'Support_group', 'category_ID', 'ID_status','Created_by', 'active', 'impact']]
df_new.shape

(141712, 8)

In [5]:
df_new = df_new[df_new.ID_status != -100]
df_new.ID_status.value_counts()
print(df_new.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141707 entries, 0 to 141711
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   count_updated  141707 non-null  int64 
 1   opened_by      141707 non-null  object
 2   Support_group  141707 non-null  object
 3   category_ID    141707 non-null  object
 4   ID_status      141707 non-null  object
 5   Created_by     141707 non-null  object
 6   active         141707 non-null  bool  
 7   impact         141707 non-null  object
dtypes: bool(1), int64(1), object(6)
memory usage: 8.8+ MB
None


In [6]:
df_new = pd.get_dummies(df_new, columns=['Support_group','category_ID','ID_status','opened_by','Created_by', 'active'], drop_first=True)
print(df_new.shape)
df_new.head()

(141707, 534)


,count_updated,impact,Support_group_Group 11,Support_group_Group 12,Support_group_Group 13,Support_group_Group 14,Support_group_Group 15,Support_group_Group 16,Support_group_Group 17,Support_group_Group 18,Support_group_Group 19,Support_group_Group 2,Support_group_Group 20,Support_group_Group 21,Support_group_Group 22,Support_group_Group 23,Support_group_Group 24,Support_group_Group 25,Support_group_Group 26,Support_group_Group 27,Support_group_Group 28,Support_group_Group 29,Support_group_Group 3,Support_group_Group 30,Support_group_Group 31,Support_group_Group 32,Support_group_Group 33,Support_group_Group 34,Support_group_Group 35,Support_group_Group 36,Support_group_Group 37,Support_group_Group 38,Support_group_Group 39,Support_group_Group 4,Support_group_Group 41,Support_group_Group 43,Support_group_Group 44,Support_group_Group 45,Support_group_Group 46,Support_group_Group 47,...,Created_by_Created by 55,Created_by_Created by 56,Created_by_Created by 57,Created_by_Created by 58,Created_by_Created by 59,Created_by_Created by 6,Created_by_Created by 60,Created_by_Created by 61,Created_by_Created by 62,Created_by_Created by 64,Created_by_Created by 65,Created_by_Created by 66,Created_by_Created by 67,Created_by_Created by 68,Created_by_Created by 69,Created_by_Created by 7,Created_by_Created by 70,Created_by_Created by 71,Created_by_Created by 73,Created_by_Created by 74,Created_by_Created by 76,Created_by_Created by 78,Created_by_Created by 79,Created_by_Created by 8,Created_by_Created by 81,Created_by_Created by 82,Created_by_Created by 83,Created_by_Created by 84,Created_by_Created by 85,Created_by_Created by 86,Created_by_Created by 87,Created_by_Created by 88,Created_by_Created by 89,Created_by_Created by 91,Created_by_Created by 92,Created_by_Created by 96,Created_by_Created by 97,Created_by_Created by 98,Created_by_Created by 99,active_True
0,0,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,4,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
from sklearn.preprocessing import MinMaxScaler
mx_scaler = MinMaxScaler()
final_df2 = mx_scaler.fit_transform(df_new.drop(columns = 'impact'))
final_df2 = pd.DataFrame(final_df2, columns=df_new.drop(columns = 'impact').columns)
print(final_df2.head())

   count_updated  Support_group_Group 11  ...  Created_by_Created by 99  active_True
0       0.000000                     0.0  ...                       0.0          1.0
1       0.023256                     0.0  ...                       0.0          1.0
2       0.031008                     0.0  ...                       0.0          0.0
3       0.007752                     0.0  ...                       0.0          1.0
4       0.015504                     0.0  ...                       0.0          1.0

[5 rows x 533 columns]


In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
pc = pca.fit_transform(final_df2)

In [9]:
final_df = pd.DataFrame(pc)
final_df['impact'] = df_new['impact']
final_df.dropna(inplace= True)
final_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,impact
0,-0.854651,-0.353830,0.022320,-0.604902,-0.139053,0.161172,0.022110,0.007168,-0.046329,0.151994,-0.088347,-0.082774,-0.105896,-0.097093,-0.168013,2 - Medium
1,-0.738597,-0.110949,-0.051450,0.749119,-0.327281,0.296415,-0.067661,0.086342,-0.026042,0.181125,-0.091369,-0.073093,-0.170319,-0.025715,-0.169768,2 - Medium
2,-0.533766,1.269262,0.248134,-0.172341,-0.048682,0.147407,-0.003544,0.030078,-0.051647,0.177850,-0.094290,-0.085769,-0.125010,-0.027035,-0.163412,2 - Medium
3,-0.797492,0.019170,0.122414,0.186887,0.234521,-0.069883,0.039002,0.046070,-0.119760,0.294898,-0.040304,-0.122939,0.265988,-0.284902,0.212774,2 - Medium
4,-0.797536,0.019288,0.122569,0.187092,0.234673,-0.069933,0.038998,0.046040,-0.119748,0.294935,-0.040328,-0.122916,0.265947,-0.284799,0.212814,2 - Medium


In [10]:
from sklearn.model_selection import train_test_split
X = final_df.drop(columns = 'impact')
y = final_df['impact']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101, stratify= y)
print(y_train.value_counts())
print(y_test.value_counts())

2 - Medium    94030
3 - Low        2717
1 - High       2444
Name: impact, dtype: int64
2 - Medium    40299
3 - Low        1165
1 - High       1047
Name: impact, dtype: int64


# Neural Networks

In [11]:
from keras.utils import to_categorical
final_df.impact = pd.Categorical(final_df.impact)
final_df.impact = final_df.impact.cat.codes
y = to_categorical(final_df.impact)

In [12]:
from keras.utils import to_categorical
final_df.impact = pd.Categorical(final_df.impact)
final_df.impact = final_df.impact.cat.codes
y = to_categorical(final_df.impact)
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(10, activation="relu",input_shape=(15,)))
model.add(Dense(10))
model.add(Dense(3, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit(X, y,validation_split=0.3, epochs = 30)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 303
Trainable params: 303
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
3100/3100 [==============================] - 4s 1ms/step - loss: 0.2248 - accuracy: 0.9463 - val_loss: 0.2509 - val_accuracy: 0.9480
Epoch 2/30
3100/3100 [==============================] - 4s 1ms/step - loss: 0.1998 - accuracy: 0.9478 - val_loss: 0.2533 - val_accuracy: 0.9478
Epoch 3/30
3100/3100 [==============================] - 4s 1ms/step - loss: 0.1981 - accuracy: 0.947

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
model = Sequential()
model.add(Dense(100, activation="relu",input_shape=(15,)))
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit(X, y,validation_split = 0.2, epochs = 100,use_multiprocessing= True, callbacks = [early_stopping])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               1600      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 22,103
Trainable params: 22,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
3543/3543 [==============================] - 5s 1ms/step - loss: 0.2144 - accuracy: 0.9458 - val_loss: 0.2293 - val_accuracy: 0.9532
Epoch 2/100
3543/3543 [==============================] - 5s 1ms/step - loss: 0.2078 - accuracy: 0.9464 